# Twitter Sentiment Analysis

**November 19th, 2018** <br>
**Austin Jarvis** 

The purpose of this project was to exhibit the ability to access the Twitter API and perform a sentiment analysis on a specific user. In this example Conan O'Brien's tweets were analyzed, his tweets were generally positive. Of the 200 tweets analyzed, 43% were positive.

In [43]:
import re 
import tweepy 
from tweepy import OAuthHandler 
from textblob import TextBlob 
from credentials import *        # To consume Twitter's API
import pandas as pd     
import numpy as np      

# For plotting and visualization:
from IPython.display import display
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

def twitter_setup():
    auth = tweepy.OAuthHandler(CONSUMER_KEY,CONSUMER_SECRET)
    auth.set_access_token(ACCESS_TOKEN,ACCESS_SECRET)
    api = tweepy.API(auth)
    return api


In [44]:
query = twitter_setup()

# We create a tweet list as follows:
tweets = query.user_timeline(screen_name="ConanOBrien", count=200)
print("Number of tweets extracted: {}.\n".format(len(tweets)))

# We print the most recent 5 tweets:
print("3 recent tweets:\n")
for tweet in tweets[:3]:
    print(tweet.text)
    print()

Number of tweets extracted: 200.

3 recent tweets:

Please, show some respect and use its full name: "Fantastic Beasts: The Crimes of Grindelwald: The Plight of Scaman… https://t.co/OdMWIjoPW7

In my experience, an apple a day keeps Hugh Jackman away, but, no, I can't confirm that they're connected.

RT @ConanOBrien: Will Ferrell sat down for the premiere episode of my new podcast. If you like Will Ferrell and Conan O’Brien but hate our…



In [45]:
# We create a pandas dataframe as follows:
data = pd.DataFrame(data=[tweet.text for tweet in tweets], columns=['Tweets'])

# We display the first 10 elements of the dataframe:
display(data.head(10))

,Tweets
0,"Please, show some respect and use its full nam..."
1,"In my experience, an apple a day keeps Hugh Ja..."
2,RT @ConanOBrien: Will Ferrell sat down for the...
3,Will Ferrell sat down for the premiere episode...
4,This is my favorite time of year: every Sunday...
5,I was just adopted by this bachelorette party ...
6,"In Texas, this is a light breakfast. #ConanAnd..."
7,This is the 47th wedding I’ve ruined. When I h...
8,Studies show young people are having less sex ...
9,"On my tour, I’m selling these limited edition ..."


In [46]:
data['len']  = np.array([len(tweet.text) for tweet in tweets])
data['ID']   = np.array([tweet.id for tweet in tweets])
data['Date'] = np.array([tweet.created_at for tweet in tweets])
data['Source'] = np.array([tweet.source for tweet in tweets])
data['Likes']  = np.array([tweet.favorite_count for tweet in tweets])
data['RTs']    = np.array([tweet.retweet_count for tweet in tweets])
display(data.head(10))

,Tweets,len,ID,Date,Source,Likes,RTs
0,"Please, show some respect and use its full nam...",140,1064661550122917888,2018-11-19 23:27:48,Hootsuite,5592,720
1,"In my experience, an apple a day keeps Hugh Ja...",106,1064591690311376898,2018-11-19 18:50:12,Hootsuite,2673,177
2,RT @ConanOBrien: Will Ferrell sat down for the...,139,1064548974927872000,2018-11-19 16:00:28,Twitter for iPhone,0,634
3,Will Ferrell sat down for the premiere episode...,140,1064503674758750208,2018-11-19 13:00:28,Hootsuite,9268,634
4,This is my favorite time of year: every Sunday...,113,1064258232473055233,2018-11-18 20:45:10,Hootsuite,23539,1722
5,I was just adopted by this bachelorette party ...,94,1064255812669972481,2018-11-18 20:35:33,Twitter for iPhone,2824,102
6,"In Texas, this is a light breakfast. #ConanAnd...",77,1063997536854249472,2018-11-18 03:29:15,Twitter for iPhone,9234,547
7,This is the 47th wedding I’ve ruined. When I h...,106,1063925254366666753,2018-11-17 22:42:02,Twitter for iPhone,9803,550
8,Studies show young people are having less sex ...,103,1063917230633742337,2018-11-17 22:10:09,Hootsuite,22858,2015
9,"On my tour, I’m selling these limited edition ...",127,1063671125908045824,2018-11-17 05:52:13,Twitter for iPhone,2779,147


In [47]:
from textblob import TextBlob
import re

def clean_tweet(tweet):
    '''
    Utility function to clean the text in a tweet by removing 
    links and special characters using regex.
    '''
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())

def analize_sentiment(tweet):
    '''
    Utility function to classify the polarity of a tweet
    using textblob.
    '''
    analysis = TextBlob(clean_tweet(tweet))
    if analysis.sentiment.polarity > 0:
        return 'positive'
    elif analysis.sentiment.polarity == 0:
        return 'neutral'
    else:
        return 'negative'

In [48]:
# We create a column with the result of the analysis:
data['SA'] = np.array([ analize_sentiment(tweet) for tweet in data['Tweets'] ])

# We display the updated dataframe with the new column:
display(data.head(10))

,Tweets,len,ID,Date,Source,Likes,RTs,SA
0,"Please, show some respect and use its full nam...",140,1064661550122917888,2018-11-19 23:27:48,Hootsuite,5592,720,positive
1,"In my experience, an apple a day keeps Hugh Ja...",106,1064591690311376898,2018-11-19 18:50:12,Hootsuite,2673,177,neutral
2,RT @ConanOBrien: Will Ferrell sat down for the...,139,1064548974927872000,2018-11-19 16:00:28,Twitter for iPhone,0,634,negative
3,Will Ferrell sat down for the premiere episode...,140,1064503674758750208,2018-11-19 13:00:28,Hootsuite,9268,634,negative
4,This is my favorite time of year: every Sunday...,113,1064258232473055233,2018-11-18 20:45:10,Hootsuite,23539,1722,positive
5,I was just adopted by this bachelorette party ...,94,1064255812669972481,2018-11-18 20:35:33,Twitter for iPhone,2824,102,neutral
6,"In Texas, this is a light breakfast. #ConanAnd...",77,1063997536854249472,2018-11-18 03:29:15,Twitter for iPhone,9234,547,positive
7,This is the 47th wedding I’ve ruined. When I h...,106,1063925254366666753,2018-11-17 22:42:02,Twitter for iPhone,9803,550,neutral
8,Studies show young people are having less sex ...,103,1063917230633742337,2018-11-17 22:10:09,Hootsuite,22858,2015,negative
9,"On my tour, I’m selling these limited edition ...",127,1063671125908045824,2018-11-17 05:52:13,Twitter for iPhone,2779,147,positive


In [49]:

pos_tweets = [ tweet for index, tweet in enumerate(data['Tweets']) if data['SA'][index] == 'positive']
neu_tweets = [ tweet for index, tweet in enumerate(data['Tweets']) if data['SA'][index] == 'negative']
neg_tweets = [ tweet for index, tweet in enumerate(data['Tweets']) if data['SA'][index] == 'negative']

print("Percentage of positive tweets: {}%".format(len(pos_tweets)*100/len(data['Tweets'])))
print("Percentage of neutral tweets: {}%".format(len(neu_tweets)*100/len(data['Tweets'])))
print("Percentage de negative tweets: {}%".format(len(neg_tweets)*100/len(data['Tweets'])))

Percentage of positive tweets: 43.0%
Percentage of neutral tweets: 19.0%
Percentage de negative tweets: 19.0%
